<!--
tags: ["runtime logging", "debugging"]
description: |
    Provide capabilities of runtime logging for debugging and performance analysis.
-->

## Runtime Logging with AutoGen 

AutoGen offers utilities to log data for debugging and performance analysis. This notebook demonstrates how to use them. 

In general, users can initiate logging by calling `autogen.runtime_logging.start()` and stop logging by calling `autogen.runtime_logging.stop()`

In [9]:
import os
import json
import autogen
from autogen import AssistantAgent, UserProxyAgent

# start logging
logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

## Create an Agent Workflow and Run it
llm_config = {
    "config_list": [
        {
            "model": "gpt-4",
            "api_key": os.environ["OPENAI_API_KEY"],  # Note, add your own API key to environment variable
        }
    ],
    "temperature": 0.9,
}

assistant = AssistantAgent(name="assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)

user_proxy.initiate_chat(
    assistant, message="What is the height of the Eiffel Tower? Only respond with the answer and terminate"
)
autogen.runtime_logging.stop()

Logging session ID: b04aa5ec-dbbc-4094-96ec-04418d6764bb
user_proxy (to assistant):

What is the height of the Eiffel Tower? Only respond with the answer and terminate

--------------------------------------------------------------------------------
assistant (to user_proxy):

The height of the Eiffel Tower is approximately 330 meters.

TERMINATE

--------------------------------------------------------------------------------


## Getting Data from the SQLite Database 

`logs.db` should be generated, by default it's using SQLite database. You can view the data with GUI tool like `sqlitebrowser`, using SQLite command line shell or using python script:



In [10]:
def get_log(dbname="logs.db", table="chat_completions"):
    import sqlite3

    con = sqlite3.connect(dbname)
    query = f"SELECT * from {table}"
    cursor = con.execute(query)
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    data = [dict(zip(column_names, row)) for row in rows]
    con.close()
    return data

In [11]:
import pandas as pd

def str_to_dict(s):
    return json.loads(s)


log_data = get_log()
log_data_df = pd.DataFrame(log_data)
log_data_df["total_tokens"] = log_data_df.apply(
    lambda row: str_to_dict(row["response"])["usage"]["total_tokens"], axis=1
)
log_data_df

,id,invocation_id,client_id,wrapper_id,session_id,request,response,is_cached,cost,start_time,end_time,total_tokens
0,1,6a6b8343-1e01-43a8-97fa-da06b84bc0e1,139932881966976,139936942273104,e5befd7d-d9ce-4b96-9c7f-ff4ee9356c7b,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8rYbYJF9Vz68vgy1SlIPrJg...",1,0.01572,2024-02-12 22:48:53.551036,2024-02-12 22:48:53.553355,507
1,2,8ee67268-2d4e-4fa3-852d-ffbdd4d82532,139936942266144,139932882206576,13a337bc-87e2-467c-9aa8-abde5676c179,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8rYbYJF9Vz68vgy1SlIPrJg...",1,0.01572,2024-02-12 22:48:55.078977,2024-02-12 22:48:55.079256,507
2,3,253362f4-5561-4817-a565-0d266c684790,139932882335440,139932882794576,b04aa5ec-dbbc-4094-96ec-04418d6764bb,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8rYbYJF9Vz68vgy1SlIPrJg...",1,0.01572,2024-02-12 22:48:56.486242,2024-02-12 22:48:56.486545,507


## Computing Cost 

One use case of telemetry data is to compute the cost of a session.

In [12]:
# sum totoal tokens for all sessions
total_tokens = log_data_df["total_tokens"].sum()

# sum total cost for all sessions
total_cost = log_data_df["cost"].sum()

# total tokens for specific session telemetry_session_id
session_tokens = log_data_df[log_data_df["session_id"] == logging_session_id]["total_tokens"].sum()
session_cost = log_data_df[log_data_df["session_id"] == logging_session_id]["cost"].sum()

print("Total tokens for all sessions: " + str(total_tokens) + ", total cost: " + str(round(total_cost, 4)))
print("Total tokens for session " + str(logging_session_id) + ": " + str(session_tokens) + ", cost: " + str(round(session_cost, 4)))

Total tokens for all sessions: 1521, total cost: 0.0472
Total tokens for session b04aa5ec-dbbc-4094-96ec-04418d6764bb: 507, cost: 0.0157
